In [1]:
import aesara
import aesara.tensor as at

from aesara import function, config

import numpy as np

In [2]:
y = at.tensor(dtype="float64", shape=(40, 30))
a = y

# We know that the shape of this should be equal to the shape of `y`
z = y[1:3, slice(1, 20, 2)]

_ = aesara.dprint(z, print_type=True)
# Subtensor{::} [id A] <TensorType(float32, (None,))> ''   
#  |<TensorType(float32, (3,))> [id B] <TensorType(float32, (3,))>

z_grad = at.grad(z.sum(), wrt=y)

Subtensor{int64:int64:, int64:int64:int64} [id A] <TensorType(float64, (2, 10))> ''   
 |<TensorType(float64, (40, 30))> [id B] <TensorType(float64, (40, 30))>
 |ScalarConstant{1} [id C] <int64>
 |ScalarConstant{3} [id D] <int64>
 |ScalarConstant{1} [id E] <int64>
 |ScalarConstant{20} [id F] <int64>
 |ScalarConstant{2} [id G] <int64>


In [3]:
z.type.shape

(2, 10)

In [4]:
aesara.dprint(z_grad)

IncSubtensor{Inc;int64:int64:, int64:int64:int64} [id A] ''   
 |Elemwise{second,no_inplace} [id B] ''   
 | |<TensorType(float64, (40, 30))> [id C]
 | |InplaceDimShuffle{x,x} [id D] ''   
 |   |TensorConstant{0.0} [id E]
 |Elemwise{second} [id F] ''   
 | |Subtensor{int64:int64:, int64:int64:int64} [id G] ''   
 | | |<TensorType(float64, (40, 30))> [id C]
 | | |ScalarConstant{1} [id H]
 | | |ScalarConstant{3} [id I]
 | | |ScalarConstant{1} [id J]
 | | |ScalarConstant{20} [id K]
 | | |ScalarConstant{2} [id L]
 | |InplaceDimShuffle{x,x} [id M] ''   
 |   |Elemwise{second,no_inplace} [id N] ''   
 |     |Sum{acc_dtype=float64} [id O] ''   
 |     | |Subtensor{int64:int64:, int64:int64:int64} [id G] ''   
 |     |TensorConstant{1.0} [id P]
 |ScalarConstant{1} [id H]
 |ScalarConstant{3} [id I]
 |ScalarConstant{1} [id J]
 |ScalarConstant{20} [id K]
 |ScalarConstant{2} [id L]


In [5]:
f_grad = function([y], z_grad)
f_grad(np.ones(shape=(40, 30)))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Test below taken from https://github.com/aesara-devs/aesara/blob/main/tests/tensor/test_basic.py#L2889-L2910

In [6]:
for inp, shape in [(at.type.scalar, ()), (at.type.vector, 3), (at.type.matrix, (3, 4))]:
    fill_value = 5
    dtype = config.floatX

    x = inp("x")
    y = at.full_like(x, fill_value, dtype=dtype)

    np.testing.assert_array_equal(
        y.eval({x: np.zeros(shape, dtype=dtype)}),
        np.full(shape, fill_value, dtype=dtype),
    )

ERROR (aesara.graph.opt): Optimization failure due to: constant_folding
ERROR (aesara.graph.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1) of 5})
ERROR (aesara.graph.opt): TRACEBACK:
ERROR (aesara.graph.opt): Traceback (most recent call last):
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1850, in process_node
    replacements = lopt.transform(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1055, in transform
    return self.fn(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/tensor/basic_opt.py", line 2954, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 131, in make_thunk
    return self.make_c_thunk(node, storage_map, compute_map, no_recycling)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 96, in make


You can find the C code in this temporary file: /var/folders/z7/rnvs_z390bbc6k_xyxktygz00000gn/T/aesara_compilation_error_hn1efcr0


ERROR (aesara.graph.opt): Optimization failure due to: constant_folding
ERROR (aesara.graph.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1) of 5})
ERROR (aesara.graph.opt): TRACEBACK:
ERROR (aesara.graph.opt): Traceback (most recent call last):
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1850, in process_node
    replacements = lopt.transform(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1055, in transform
    return self.fn(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/tensor/basic_opt.py", line 2954, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 131, in make_thunk
    return self.make_c_thunk(node, storage_map, compute_map, no_recycling)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 96, in make


You can find the C code in this temporary file: /var/folders/z7/rnvs_z390bbc6k_xyxktygz00000gn/T/aesara_compilation_error_5z6pqda1


ERROR (aesara.graph.opt): Optimization failure due to: constant_folding
ERROR (aesara.graph.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1) of 5})
ERROR (aesara.graph.opt): TRACEBACK:
ERROR (aesara.graph.opt): Traceback (most recent call last):
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1850, in process_node
    replacements = lopt.transform(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1055, in transform
    return self.fn(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/tensor/basic_opt.py", line 2954, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 131, in make_thunk
    return self.make_c_thunk(node, storage_map, compute_map, no_recycling)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 96, in make

...

ERROR (aesara.graph.opt): Optimization failure due to: constant_folding
ERROR (aesara.graph.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1) of 5})
ERROR (aesara.graph.opt): TRACEBACK:
ERROR (aesara.graph.opt): Traceback (most recent call last):
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1850, in process_node
    replacements = lopt.transform(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1055, in transform
    return self.fn(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/tensor/basic_opt.py", line 2954, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 131, in make_thunk
    return self.make_c_thunk(node, storage_map, compute_map, no_recycling)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 96, in make


You can find the C code in this temporary file: /var/folders/z7/rnvs_z390bbc6k_xyxktygz00000gn/T/aesara_compilation_error_fdwr84cx


ERROR (aesara.graph.opt): Optimization failure due to: constant_folding
ERROR (aesara.graph.opt): node: Elemwise{Cast{float64}}(TensorConstant{(1, 1) of 5})
ERROR (aesara.graph.opt): TRACEBACK:
ERROR (aesara.graph.opt): Traceback (most recent call last):
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1850, in process_node
    replacements = lopt.transform(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/graph/opt.py", line 1055, in transform
    return self.fn(fgraph, node)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/tensor/basic_opt.py", line 2954, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 131, in make_thunk
    return self.make_c_thunk(node, storage_map, compute_map, no_recycling)
  File "/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/op.py", line 96, in make


You can find the C code in this temporary file: /var/folders/z7/rnvs_z390bbc6k_xyxktygz00000gn/T/aesara_compilation_error_j0c_ibo7

You can find the C code in this temporary file: /var/folders/z7/rnvs_z390bbc6k_xyxktygz00000gn/T/aesara_compilation_error_0mgat91i


CompileError: Compilation failed (return status=1):
/Users/larryshamalama/opt/anaconda3/envs/aesara-dev/bin/clang++ -dynamiclib -g -O3 -fno-math-errno -Wno-unused-label -Wno-unused-variable -Wno-write-strings -march=westmere -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -fPIC -undefined dynamic_lookup -I/Users/larryshamalama/opt/anaconda3/envs/aesara-dev/lib/python3.10/site-packages/numpy/core/include -I/Users/larryshamalama/opt/anaconda3/envs/aesara-dev/include/python3.10 -I/Users/larryshamalama/Documents/GitHub/aesara/aesara/link/c/c_code -L/Users/larryshamalama/opt/anaconda3/envs/aesara-dev/lib -fvisibility=hidden -o /Users/larryshamalama/.aesara/compiledir_macOS-12.2.1-x86_64-i386-64bit-i386-3.10.4-64/tmpr220553e/m7b2bdf352fbc758c4bc8a462fb839e3b2f56abd0e5d9776aada223d38c98b23c.so /Users/larryshamalama/.aesara/compiledir_macOS-12.2.1-x86_64-i386-64bit-i386-3.10.4-64/tmpr220553e/mod.cpp
/Users/larryshamalama/.aesara/compiledir_macOS-12.2.1-x86_64-i386-64bit-i386-3.10.4-64/tmpr220

In [ ]:
y.eval({x: np.zeros(shape, dtype=dtype)})

In [ ]:
# A vector of length three built from a `MakeVector` `Op`
x = at.as_tensor([at.lscalar(), at.lscalar(), at.lscalar()])
z = x[-2:]

aesara.dprint(z, print_type=True)
# Subtensor{int64::} [id A] <TensorType(int64, (5,))> ''
#  |MakeVector{dtype='int64'} [id B] <TensorType(int64, (3,))> ''
#  |ScalarConstant{-2} [id C] <int64>